## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pisco,PE,-13.7000,-76.2167,73.40,78,0,21.85,clear sky
1,1,Tuatapere,NZ,-46.1333,167.6833,57.00,74,30,8.99,scattered clouds
2,2,Belyy Yar,RU,53.6039,91.3903,19.40,58,0,11.18,clear sky
3,3,Grindavik,IS,63.8424,-22.4338,19.40,73,0,4.61,clear sky
4,4,Portland,US,45.5234,-122.6762,46.99,87,90,6.91,light intensity drizzle


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Atuona,PF,-9.8000,-139.0333,77.79,80,100,17.18,overcast clouds
23,23,Rikitea,PF,-23.1203,-134.9692,78.30,76,89,13.78,overcast clouds
25,25,Meulaboh,ID,4.1363,96.1285,75.45,89,100,4.38,overcast clouds
27,27,Hithadhoo,MV,-0.6000,73.0833,80.89,76,100,15.28,overcast clouds
32,32,Kirakira,SB,-10.4544,161.9205,83.73,75,98,5.10,light rain
33,33,Mar Del Plata,AR,-38.0023,-57.5575,77.00,65,0,23.02,clear sky
39,39,Senador Jose Porfirio,BR,-2.5908,-51.9542,84.22,71,74,3.02,light rain
51,51,Latung,PH,5.5000,120.8833,83.10,72,94,4.36,overcast clouds
55,55,Alice Springs,AU,-23.7000,133.8833,77.00,64,6,4.61,clear sky
56,56,Orocue,CO,4.7912,-71.3354,79.00,80,96,6.67,heavy intensity rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                182
City                   182
Country                181
Lat                    182
Lng                    182
Max Temp               182
Humidity               182
Cloudiness             182
Wind Speed             182
Current Description    182
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                181
City                   181
Country                181
Lat                    181
Lng                    181
Max Temp               181
Humidity               181
Cloudiness             181
Wind Speed             181
Current Description    181
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Atuona,PF,77.79,overcast clouds,-9.8000,-139.0333,
23,Rikitea,PF,78.30,overcast clouds,-23.1203,-134.9692,
25,Meulaboh,ID,75.45,overcast clouds,4.1363,96.1285,
27,Hithadhoo,MV,80.89,overcast clouds,-0.6000,73.0833,
32,Kirakira,SB,83.73,light rain,-10.4544,161.9205,
33,Mar Del Plata,AR,77.00,clear sky,-38.0023,-57.5575,
39,Senador Jose Porfirio,BR,84.22,light rain,-2.5908,-51.9542,
51,Latung,PH,83.10,overcast clouds,5.5000,120.8833,
55,Alice Springs,AU,77.00,clear sky,-23.7000,133.8833,
56,Orocue,CO,79.00,heavy intensity rain,4.7912,-71.3354,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"  
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #abc = hotels["results"][0]["name"]
        #print(abc)
    except (IndexError):
        print("Hotel not found... skipping.")   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
#Replace the empty string with NAN
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Atuona,PF,77.79,overcast clouds,-9.8000,-139.0333,Villa Enata
23,Rikitea,PF,78.30,overcast clouds,-23.1203,-134.9692,Pension Maro'i
25,Meulaboh,ID,75.45,overcast clouds,4.1363,96.1285,Kost Pria
27,Hithadhoo,MV,80.89,overcast clouds,-0.6000,73.0833,Scoop Guest House
32,Kirakira,SB,83.73,light rain,-10.4544,161.9205,SanBiz Lodge
33,Mar Del Plata,AR,77.00,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
39,Senador Jose Porfirio,BR,84.22,light rain,-2.5908,-51.9542,Pousada Novo Horizonte
51,Latung,PH,83.10,overcast clouds,5.5000,120.8833,NaN
55,Alice Springs,AU,77.00,clear sky,-23.7000,133.8833,Desert Palms Alice Springs
56,Orocue,CO,79.00,heavy intensity rain,4.7912,-71.3354,Hotel Brisas de San Miguel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Atuona,PF,77.79,overcast clouds,-9.8000,-139.0333,Villa Enata
23,Rikitea,PF,78.30,overcast clouds,-23.1203,-134.9692,Pension Maro'i
25,Meulaboh,ID,75.45,overcast clouds,4.1363,96.1285,Kost Pria
27,Hithadhoo,MV,80.89,overcast clouds,-0.6000,73.0833,Scoop Guest House
32,Kirakira,SB,83.73,light rain,-10.4544,161.9205,SanBiz Lodge
33,Mar Del Plata,AR,77.00,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
39,Senador Jose Porfirio,BR,84.22,light rain,-2.5908,-51.9542,Pousada Novo Horizonte
55,Alice Springs,AU,77.00,clear sky,-23.7000,133.8833,Desert Palms Alice Springs
56,Orocue,CO,79.00,heavy intensity rain,4.7912,-71.3354,Hotel Brisas de San Miguel
58,Hilo,US,78.80,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))